In [12]:
pip install pandas geopandas folium branca

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 75.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 81.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import pandas as pd
import geopandas as gpd
import folium
from branca.colormap import linear

# ———— 配置 ————
CSV_PATH    = "transactions.csv"
KML_PATH    = "sample_uk_postcode_area.kml"
OUTPUT_HTML = "transaction_heatmap.html"

# ———— 1. 读取 CSV 并汇总交易金额 ————
# CSV 内部字段：ad_gb_postcode_outcode, txn_amount
df_txn = pd.read_csv(
    CSV_PATH,
    dtype={"ad_gb_postcode_outcode": str, "txn_amount": float}
)
# 按 outcode 聚合
df_txn = (
    df_txn
    .groupby("ad_gb_postcode_outcode", as_index=False)
    .agg({"txn_amount": "sum"})
    .rename(columns={"ad_gb_postcode_outcode": "postcode"})
)

# ———— 2. 读取 KML，提取 postcode 区域边界 ————
# KML 中每个 Placemark 的 <name> 存放 outcode
gdf = (
    gpd.read_file(KML_PATH, driver="KML")
       .loc[:, ["Name", "geometry"]]
       .rename(columns={"Name": "postcode"})
)

# ———— 3. 合并边界与数据 ————
gdf = gdf.merge(df_txn, on="postcode", how="left")
gdf["txn_amount"] = gdf["txn_amount"].fillna(0.0)

# ———— 4. 构造色阶 from 0 to max(txn_amount) ————
max_amt = gdf["txn_amount"].max()
colormap = linear.YlGn_09.scale(0, max_amt)
colormap.caption = "Transaction Amount by Postcode Outcode"

# ———— 5. 初始化 Folium 地图 ————
m = folium.Map(
    location=[54.0, -2.0],
    zoom_start=5,
    tiles="cartodbpositron"
)

# ———— 6. 添加 Choropleth 图层 ————
folium.Choropleth(
    geo_data=gdf.__geo_interface__,
    name="choropleth",
    data=gdf,
    columns=["postcode", "txn_amount"],
    key_on="feature.properties.postcode",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Txn Amount"
).add_to(m)

# ———— 7. 为每个区域添加鼠标悬停提示 ————
for _, row in gdf.iterrows():
    style = {
        "fillColor": colormap(row["txn_amount"]),
        "color": "#529",
        "weight": 0.5,
        "fillOpacity": 0.7
    }
    gj = folium.GeoJson(
        row["geometry"],
        style_function=lambda feat, style=style: style,
        tooltip=folium.Tooltip(
            f"{row['postcode']}: £{row['txn_amount']:,.2f}"
        )
    )
    gj.add_to(m)

# ———— 8. 添加色标控件 & 保存 ————
colormap.add_to(m)
m.save(OUTPUT_HTML)

print(f"✅ 热力图已生成：{os.path.abspath(OUTPUT_HTML)}")


✅ 热力图已生成：/workspaces/FTP-Exercise/transaction_heatmap.html


print(1+1)

In [1]:
print(1+1)

2
